# Pipeline Completo de Corte/Vinco - Machine Learning

Este notebook executa o pipeline completo end-to-end de machine learning para máquinas de Corte/Vinco, incluindo:

1. **Carregamento de Dados** - Dados de produção e pedidos
2. **Feature Engineering** - Criação de features geométricas
3. **Clusterização GMM** - Identificação de produtos similares
4. **Treinamento do Modelo** - Classificação de produtividade
5. **Avaliação** - Métricas e feature importance
6. **Explicabilidade** - SHAP values (opcional)
7. **Inferência** - Predição em novos pedidos

**✅ Correções Aplicadas:**
- Import de `Iterable` em clustering.py
- Uso correto de `np.nan` ao invés de `pd.NA`
- Construção correta da feature matrix sem duplicação de `PROB_CLUSTER_*`
- Lógica simplificada de alinhamento de features na inferência

---

In [ ]:
# === CONFIGURAÇÃO DO AMBIENTE ===

import sys
from pathlib import Path
import pandas as pd
import numpy as np
from pprint import pprint


# Encontrar raiz do projeto
def find_project_root(marker="pyproject.toml"):
    """Retorna o diretório do projeto procurando pelo arquivo marcador."""
    current = Path.cwd().resolve()
    for path in (current,) + tuple(current.parents):
        if (path / marker).exists():
            return path
    return current


PROJECT_ROOT = find_project_root()
SRC_DIR = PROJECT_ROOT / "src"
DATA_DIR = PROJECT_ROOT / "data"

if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Importar módulos do pipeline
from pipelines.DS import pipelines, inference, data_processing

print(f"✓ Projeto: {PROJECT_ROOT}")
print(f"✓ Source: {SRC_DIR}")
print(f"✓ Data: {DATA_DIR}")

✓ Projeto: /home/adami/Projeto_IA_AMCOM/project_data_science
✓ Source: /home/adami/Projeto_IA_AMCOM/project_data_science/src
✓ Data: /home/adami/Projeto_IA_AMCOM/project_data_science/data


## 1. Executar Pipeline Completo

O pipeline `run_corte_pipeline` executa todas as etapas automaticamente:
- Carregamento de dados (produção + pedidos + facas)
- Feature engineering
- Clusterização GMM (seleção automática de K via BIC)
- Feature selection
- Treinamento do modelo
- Avaliação e explicabilidade

In [5]:
# Defina o caminho para o arquivo de pedidos brutos
raw_pedidos_path = (
    "/home/adami/Projeto_IA_AMCOM/project_data_science/data/raw/tb_pedidos.parquet"
)

# Exemplo de carregamento (ajuste conforme sua estrutura de diretórios)
pedidos_df = pd.read_parquet(raw_pedidos_path)


In [6]:
pedidos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406890 entries, 0 to 406889
Data columns (total 87 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   CD_PEDIDO                   406890 non-null  string        
 1   CD_ITEM                     406890 non-null  string        
 2   ID_CLIENTE                  406890 non-null  string        
 3   CD_PALETE                   406890 non-null  string        
 4   CD_TIPOFT2                  406890 non-null  string        
 5   CD_REFERENCIA               406890 non-null  string        
 6   DT_ENTREGA2                 406890 non-null  datetime64[ns]
 7   DT_ENTREGAORIGINAL          406890 non-null  datetime64[ns]
 8   FL_AMARRADO                 406890 non-null  string        
 9   FL_CHAPA                    406890 non-null  string        
 10  FL_EXIGELAUDO               406890 non-null  string        
 11  FL_PALETIZADO               406890 non-

In [7]:
# Colunas a excluir
colunas_excluir = [
    # Identificadores
    "CD_OP",
    "CD_ITEM",
    "CD_PEDIDO",
    "CD_FACA",
    # Variáveis target (resultados de produção)
    "QT_PRODUZIDA",
    "QT_CHAPASALIMENTADAS",
    "VL_DURACAO_PRODUCAO",
    "QT_PROGRAMADA",
    "VL_DURACAO_PARADAS",
    "QT_PARADAS",
    # Categóricas de alta cardinalidade (analisar DEPOIS do clustering)
    "CAT_COMPOSICAO",
    "TX_TIPOABNT",
    "VL_MULTCOMP",
    "VL_MULTLARG",
    "FL_REFILADO",
    "PECAS_POR_CHAPA",
    "QT_ARRANJO",
    # Redundante com VL_COMPRIMENTO
    # "VL_COMPRIMENTO",  # 0.9999 correlação
    # Redundante com VL_LARGURA
    # "VL_LARGURA",  # 0.9999 correlação
    # Redundante com RAZAO_COMP_LARG
    "RAZAO_PECA_COMP_LARG",  # 0.9999 correlação
    "RAZAO_CHAPA_COMP_LARG",  # 0.9999 correlação
    # Redundante com AREA_CHAPA ou VL_AREALIQUIDAPECA
    # 'AREA_CHAPA',  # Manter uma das duas
    "VL_AREALIQUIDAPECA",  # Remover (AREA_CHAPA é mais interpretável)
    # # Vincos individuais - usar só VL_VINCOS_TOTAL_MM
    "VL_VINCOCOMP1",
    "VL_VINCOCOMP2",
    "VL_VINCOCOMP3",
    "VL_VINCOCOMP4",
    "VL_VINCOCOMP5",
    "VL_VINCOLARG1",
    "VL_VINCOLARG2",
    "VL_VINCOLARG3",
    # Redundante com VL_REFILOCOMPRIMENTO
    "REFILO_TOTAL",
    "AREA_CHAPA",
    "VL_VINCOS_TOTAL_MM",
    "VL_COMPRIMENTOINTERNO",
    "VL_ALTURAINTERNA",
    "VL_LARGURAINTERNA",
    "PECAS_POR_CHAPA",
    "VL_PESOPECA",
    "VL_REFILOCOMPRIMENTO",
    "VL_PESOCAIXA",
    "QT_PEDIDA",
]




In [9]:
print("=" * 80)
print("EXECUTANDO PIPELINE DE CORTE/VINCO COM CATEGÓRICAS")
print("=" * 80 + "\n")
# Colunas a excluir
colunas_excluir = [
    # Identificadores
    "CD_OP",
    "CD_ITEM",
    "CD_PEDIDO",
    "CD_FACA",
    # Variáveis target (resultados de produção)
    "QT_PRODUZIDA",
    "QT_CHAPASALIMENTADAS",
    "VL_DURACAO_PRODUCAO",
    "QT_PROGRAMADA",
    "VL_DURACAO_PARADAS",
    "QT_PARADAS",
    # Categóricas de alta cardinalidade (analisar DEPOIS do clustering)
    "CAT_COMPOSICAO",
    "TX_TIPOABNT",
    "VL_MULTCOMP",
    "VL_MULTLARG",
    "FL_REFILADO",
    "PECAS_POR_CHAPA",
    "QT_ARRANJO",
    # Redundante com VL_COMPRIMENTO
    # "VL_COMPRIMENTO",  # 0.9999 correlação
    # Redundante com VL_LARGURA
    # "VL_LARGURA",  # 0.9999 correlação
    # Redundante com RAZAO_COMP_LARG
    "RAZAO_PECA_COMP_LARG",  # 0.9999 correlação
    "RAZAO_CHAPA_COMP_LARG",  # 0.9999 correlação
    # Redundante com AREA_CHAPA ou VL_AREALIQUIDAPECA
    'AREA_CHAPA',  # Manter uma das duas
    "VL_AREALIQUIDAPECA",  # Remover (AREA_CHAPA é mais interpretável)
    # # Vincos individuais - usar só VL_VINCOS_TOTAL_MM
    "VL_VINCOCOMP1",
    "VL_VINCOCOMP2",
    "VL_VINCOCOMP3",
    "VL_VINCOCOMP4",
    "VL_VINCOCOMP5",
    "VL_VINCOLARG1",
    "VL_VINCOLARG2",
    "VL_VINCOLARG3",
    # Redundante com VL_REFILOCOMPRIMENTO
    "REFILO_TOTAL",
    "AREA_CHAPA",
    "VL_VINCOS_TOTAL_MM",
    "VL_COMPRIMENTOINTERNO",
    "VL_ALTURAINTERNA",
    "VL_LARGURAINTERNA",
    "PECAS_POR_CHAPA",
    "VL_PESOPECA",
    "VL_REFILOCOMPRIMENTO",
    "VL_PESOCAIXA",
    "QT_PEDIDA",
]


# Executar pipeline completo
resultados = pipelines.run_flexo_pipeline(
    # === DADOS ===
    pedidos_df=None,  # None = carregar automaticamente do arquivo raw
    pedidos_cutoff="2024-01-01",  # Filtrar pedidos >= 2024-01-01
    # === CLUSTERIZAÇÃO ===
    cluster_k=4,  # None = selecionar automaticamente via BIC
    # cluster_range=range(2, 11),  # Testar K de 2 a 10
    # === FEATURES ===
    # IMPORTANTE: Usar lista personalizada que INCLUI CAT_COMPOSICAO e TX_TIPOABNT
    exclude_features=colunas_excluir,
    # === TARGET ===
    productivity_quantile=0.6,  # Threshold: 60% das operações mais produtivas
    classification_threshold=0.7,  # Threshold de probabilidade para classe positiva
    # === FEATURE SELECTION ===
    # Usar "correlation" primeiro para remover redundância numérica
    # Features categóricas não serão afetadas
    feature_selection_method="tree",
    feature_selection_params={"threshold": 0.05},
    # === MODELO ===
    # CatBoost é IDEAL para features categóricas!
    model_type="catboost",
    # === EXPLICABILIDADE ===
    shap_sample_size=0,  # DESABILITADO temporariamente
    # === REPRODUTIBILIDADE ===
    random_state=42,
    save_model=True,
    model_name="flexo_model_artifacts",
)

print("\n" + "=" * 80)
print("PIPELINE CONCLUÍDO COM SUCESSO!")
print("=" * 80)
print(f"\nFeatures selecionadas: {len(resultados['selected_features'])}")
print(
    f"   - Incluindo CAT_COMPOSICAO: {'CAT_COMPOSICAO' in resultados['selected_features']}"
)
print(f"   - Incluindo TX_TIPOABNT: {'TX_TIPOABNT' in resultados['selected_features']}")


EXECUTANDO PIPELINE DE CORTE/VINCO COM CATEGÓRICAS

Model artifacts saved to: /home/adami/Projeto_IA_AMCOM/project_data_science/src/model/flexo_flexo_model_artifacts_20251117_012637
Streamlit-compatible model saved to: /home/adami/Projeto_IA_AMCOM/project_data_science/src/model/flexo_model_artifacts.pkl

PIPELINE CONCLUÍDO COM SUCESSO!

Features selecionadas: 19
   - Incluindo CAT_COMPOSICAO: False
   - Incluindo TX_TIPOABNT: False


## 2. Informações do Dataset

In [ ]:
print("\n DATASET")
print("-" * 80)
print(f"Total de registros: {len(resultados['df']):,}")
print(f"Features totais: {len(resultados['df'].columns)}")
print(f"Features selecionadas: {len(resultados['selected_features'])}")
print(f"Features excluídas: {len(resultados['exclude_features'])}")


 DATASET
--------------------------------------------------------------------------------
Total de registros: 78,684
Features totais: 71
Features selecionadas: 20
Features excluídas: 29


In [ ]:
# Target distribution
target_dist = resultados["df"]["y_produtivo"].value_counts().sort_index()
print(f"\nDistribuição do target (y_produtivo):")
for val, count in target_dist.items():
    label = "Não produtivo" if val == 0 else "Produtivo"
    print(f"  {label:15s}: {count:6,} ({count / len(resultados['df']) * 100:5.1f}%)")



Distribuição do target (y_produtivo):
  Não produtivo  : 47,193 ( 60.0%)
  Produtivo      : 31,488 ( 40.0%)


In [ ]:



# Mostrar primeiras linhas do dataset
print("\nPrimeiras 5 linhas do dataset processado:")
resultados["df"].head()



Primeiras 5 linhas do dataset processado:


,CD_OP,QT_PRODUZIDA,QT_CHAPASALIMENTADAS,VL_DURACAO_PRODUCAO,QT_PROGRAMADA,VL_DURACAO_PARADAS,QT_PARADAS,CD_PEDIDO,CD_ITEM,FL_CONTROLE_ESPECIAL_IMPRESSAO,...,RAZAO_INTERNA,AREA_CHAPA,PECAS_POR_CHAPA,REFILO_TOTAL,CLUSTER_ID,y_produtivo,PROB_CLUSTER_0,PROB_CLUSTER_1,PROB_CLUSTER_2,PROB_CLUSTER_3
0,630166-9/518441,4200.0,2100.0,65.0,4200.0,41.0,4.0,630166-9,518441,0,...,1.333333,1346589.0,2.0,35.0,2,0.0,4.196267e-49,0.0,1.0,3.837098e-18
1,630384-9/797231,4800.0,1600.0,48.0,4800.0,26.0,4.0,630384-9,797231,0,...,1.418060,1395385.0,3.0,25.0,2,0.0,5.182819e-46,0.0,1.0,1.397898e-17
2,632404-65/518441,4200.0,2100.0,36.0,4200.0,12.0,1.0,632404-65,518441,0,...,1.333333,1346589.0,2.0,35.0,2,0.0,4.196267e-49,0.0,1.0,3.837098e-18
3,634941-27/826030,8700.0,2202.0,55.0,8704.0,32.0,3.0,634941-27,826030,0,...,1.850794,1491860.0,4.0,35.0,2,0.0,2.866302e-37,0.0,1.0,1.575677e-13
4,634941-45/826030,8600.0,2192.0,90.0,8704.0,69.0,5.0,634941-45,826030,0,...,1.850794,1491860.0,4.0,35.0,2,0.0,2.866302e-37,0.0,1.0,1.575677e-13


## 3. Resultados da Clusterização (GMM)

In [ ]:
print("\n CLUSTERIZAÇÃO")
print("-" * 80)
print(f"Número de clusters (K): {resultados['cluster_k']}")
print(f"Componentes PCA: {resultados['clustering_artifacts'].X_pca.shape[1]}")
print(
    f"Variância explicada (PCA): {resultados['clustering_artifacts'].pca.explained_variance_ratio_.sum():.2%}"
)

# Distribuição dos clusters
cluster_dist = (
    pd.Series(resultados["clustering_artifacts"].labels).value_counts().sort_index()
)
print(f"\nDistribuição dos clusters:")
for cluster_id, count in cluster_dist.items():
    print(
        f"  Cluster {cluster_id}: {count:6,} ({count / len(resultados['df']) * 100:5.1f}%)"
    )



 CLUSTERIZAÇÃO
--------------------------------------------------------------------------------
Número de clusters (K): 4
Componentes PCA: 26
Variância explicada (PCA): 95.64%

Distribuição dos clusters:
  Cluster 0:  5,881 (  7.5%)
  Cluster 1:    150 (  0.2%)
  Cluster 2: 62,380 ( 79.3%)
  Cluster 3: 10,273 ( 13.1%)


In [ ]:

# Estatísticas por cluster
print(f"\nEstatísticas básicas por cluster:")
cluster_stats = (
    resultados["df"].groupby("CLUSTER_ID")["y_produtivo"].agg(["count", "mean"])
)
cluster_stats.columns = ["Total", "Taxa_Produtividade"]
cluster_stats



Estatísticas básicas por cluster:


,Total,Taxa_Produtividade
CLUSTER_ID,,
0,5881,0.585785
1,150,0.006667
2,62378,0.404373
3,10272,0.274338


## 4. Métricas do Modelo

In [ ]:
metrics = resultados["metrics"]

print("\n MODELO DE CLASSIFICAÇÃO")
print("-" * 80)
print(f"Modelo: Hist Gradient Boosting Classifier")
print(f"ROC AUC: {metrics['roc_auc']:.4f}")
print(f"Threshold de classificação: {metrics['threshold']}")

# Classification report detalhado
report = metrics["classification_report"]

print("\n Métricas por classe:")
print(f"\n  Classe 0 (Não Produtivo):")
print(f"    Precision: {report['0']['precision']:.4f}")
print(f"    Recall:    {report['0']['recall']:.4f}")
print(f"    F1-score:  {report['0']['f1-score']:.4f}")
print(f"    Support:   {report['0']['support']:,}")

print(f"\n  Classe 1 (Produtivo):")
print(f"    Precision: {report['1']['precision']:.4f}")
print(f"    Recall:    {report['1']['recall']:.4f}")
print(f"    F1-score:  {report['1']['f1-score']:.4f}")
print(f"    Support:   {report['1']['support']:,}")

print(f"\n Métricas gerais:")
print(f"    Accuracy:       {report['accuracy']:.4f}")
print(f"    Macro avg F1:   {report['macro avg']['f1-score']:.4f}")
print(f"    Weighted avg F1: {report['weighted avg']['f1-score']:.4f}")



 MODELO DE CLASSIFICAÇÃO
--------------------------------------------------------------------------------
Modelo: Hist Gradient Boosting Classifier
ROC AUC: 0.8645
Threshold de classificação: 0.7

 Métricas por classe:

  Classe 0 (Não Produtivo):
    Precision: 0.7179
    Recall:    0.9050
    F1-score:  0.8006
    Support:   9,439.0

  Classe 1 (Produtivo):
    Precision: 0.7663
    Recall:    0.4670
    F1-score:  0.5803
    Support:   6,298.0

 Métricas gerais:
    Accuracy:       0.7297
    Macro avg F1:   0.6905
    Weighted avg F1: 0.7125


## 5. Feature Importance (Permutation)


In [ ]:

print("\n⚡ FEATURE IMPORTANCE")
print("-" * 80)
print("Top 20 features mais importantes (Permutation Importance):\n")

top_20 = resultados["feature_importance"].head(20)
for idx, (feature, importance) in enumerate(top_20.items(), 1):
    # Adicionar barra de progresso visual
    bar_length = int(importance * 50 / top_20.max())
    bar = "█" * bar_length + "░" * (50 - bar_length)
    print(f"{idx:2d}. {feature:45s} {importance:8.5f} {bar}")

# Criar DataFrame para visualização
importance_df = pd.DataFrame(
    {
        "feature": resultados["feature_importance"].index,
        "importance": resultados["feature_importance"].values,
    }
).head(50)

importance_df



⚡ FEATURE IMPORTANCE
--------------------------------------------------------------------------------
Top 20 features mais importantes (Permutation Importance):

 1. VL_COMPPECA                                    0.05914 ██████████████████████████████████████████████████
 2. VL_COMPRIMENTO                                 0.03565 ██████████████████████████████░░░░░░░░░░░░░░░░░░░░
 3. VL_LARGPECA                                    0.01557 █████████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
 4. VL_REFUGOCLIENTE                               0.01331 ███████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
 5. VL_CONSUMO_COR_TOTAL                           0.00945 ███████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
 6. QT_NRCORES                                     0.00548 ████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
 7. PROB_CLUSTER_3                                 0.00417 ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
 8. PERC_VAR_PEDIDA                                0.00411 ███░░░░░

,feature,importance
0,VL_COMPPECA,0.059140
1,VL_COMPRIMENTO,0.035649
2,VL_LARGPECA,0.015568
3,VL_REFUGOCLIENTE,0.013314
4,VL_CONSUMO_COR_TOTAL,0.009455
5,QT_NRCORES,0.005479
6,PROB_CLUSTER_3,0.004174
7,PERC_VAR_PEDIDA,0.004112
8,RAZAO_COMP_LARG,0.003941
9,PROB_CLUSTER_0,0.003264


## 6. Explicabilidade (SHAP) - Opcional

In [ ]:

if resultados["shap_values"] is not None:
    print("\n EXPLICABILIDADE (SHAP)")
    print("-" * 80)
    print(f"SHAP values calculados: {resultados['shap_values'].shape}")
    print(f"Top features por amostra: {len(resultados['top_features'])} amostras")

    # Mostrar exemplo de explicação
    print("\nExemplo de explicação (primeiras 3 amostras):")
    print(resultados["top_features"].head(3))
else:
    print("\nSHAP não foi executado (shap_sample_size=0 ou SHAP não instalado)")



SHAP não foi executado (shap_sample_size=0 ou SHAP não instalado)


## 7. Salvar Resultados


In [ ]:
import pickle
import json

output_dir = DATA_DIR / "ml" / "corte_vinco_results"
output_dir.mkdir(parents=True, exist_ok=True)

print("\n SALVANDO RESULTADOS")
print("-" * 80)

# 1. DataFrame com clusters e predições
df_output = resultados["df"].copy()
output_file = output_dir / "df_with_clusters_and_predictions.parquet"
df_output.to_parquet(output_file, index=False)
print(f"✓ DataFrame salvo: {output_file.name}")
print(f"  Registros: {len(df_output):,}")
print(f"  Colunas: {len(df_output.columns)}")



 SALVANDO RESULTADOS
--------------------------------------------------------------------------------
✓ DataFrame salvo: df_with_clusters_and_predictions.parquet
  Registros: 78,684
  Colunas: 71


In [ ]:
# 2. Feature importance
importance_file = output_dir / "feature_importance.csv"
resultados["feature_importance"].to_csv(importance_file)
print(f"\n✓ Feature importance salvo: {importance_file.name}")



✓ Feature importance salvo: feature_importance.csv


In [ ]:
# 3. Métricas
metrics_file = output_dir / "metrics.json"
with open(metrics_file, "w") as f:
    json.dump(
        {
            "roc_auc": metrics["roc_auc"],
            "threshold": metrics["threshold"],
            "classification_report": metrics["classification_report"],
            "cluster_k": resultados["cluster_k"],
            "n_features_selected": len(resultados["selected_features"]),
            "n_features_excluded": len(resultados["exclude_features"]),
            "n_samples": len(resultados["df"]),
            "pca_variance_explained": float(
                resultados["clustering_artifacts"].pca.explained_variance_ratio_.sum()
            ),
            "n_pca_components": resultados["clustering_artifacts"].X_pca.shape[1],
        },
        f,
        indent=2,
    )
print(f"\n✓ Métricas salvas: {metrics_file.name}")



✓ Métricas salvas: metrics.json


In [ ]:
metrics


{'roc_auc': 0.8644759428855592,
 'classification_report': {'0': {'precision': 0.7178754517186318,
   'recall': 0.904968746689268,
   'f1-score': 0.8006373605773737,
   'support': 9439.0},
  '1': {'precision': 0.7662845231891611,
   'recall': 0.466973642426167,
   'f1-score': 0.5803078137332282,
   'support': 6298.0},
  'accuracy': 0.7296816419902141,
  'macro avg': {'precision': 0.7420799874538964,
   'recall': 0.6859711945577175,
   'f1-score': 0.6904725871553009,
   'support': 15737.0},
  'weighted avg': {'precision': 0.7372489239256212,
   'recall': 0.7296816419902141,
   'f1-score': 0.7124607394917519,
   'support': 15737.0}},
 'threshold': 0.7}

In [ ]:
# 4. Features selecionadas
features_file = output_dir / "selected_features.txt"
with open(features_file, "w") as f:
    for feature in resultados["selected_features"]:
        f.write(f"{feature}\n")
print(f"\n✓ Features selecionadas salvas: {features_file.name}")
print(f"  Total: {len(resultados['selected_features'])}")



✓ Features selecionadas salvas: selected_features.txt
  Total: 20


In [ ]:
# 5. Artefatos do pipeline (para inferência)
artifacts_file = output_dir / "pipeline_artifacts.pkl"
with open(artifacts_file, "wb") as f:
    pickle.dump(
        {
            "clustering_artifacts": resultados["clustering_artifacts"],
            "gmm": resultados["gmm"],
            "classifier": resultados["classifier"],
            "cluster_k": resultados["cluster_k"],
            "selected_features": resultados["selected_features"],
            "exclude_features": resultados["exclude_features"],
            "metrics": resultados["metrics"],
        },
        f,
    )
print(f"\n✓ Artefatos do pipeline salvos: {artifacts_file.name}")
print(f"  (Use para inferência em novos dados)")

print(f"\n Todos os resultados salvos em: {output_dir}")
print("=" * 80)



✓ Artefatos do pipeline salvos: pipeline_artifacts.pkl
  (Use para inferência em novos dados)

 Todos os resultados salvos em: /home/adami/Projeto_IA_AMCOM/project_data_science/data/ml/corte_vinco_results


In [ ]:
resultados["df"]

,CD_OP,QT_PRODUZIDA,QT_CHAPASALIMENTADAS,VL_DURACAO_PRODUCAO,QT_PROGRAMADA,VL_DURACAO_PARADAS,QT_PARADAS,CD_PEDIDO,CD_ITEM,FL_CONTROLE_ESPECIAL_IMPRESSAO,...,RAZAO_INTERNA,AREA_CHAPA,PECAS_POR_CHAPA,REFILO_TOTAL,CLUSTER_ID,y_produtivo,PROB_CLUSTER_0,PROB_CLUSTER_1,PROB_CLUSTER_2,PROB_CLUSTER_3
0,630166-9/518441,4200.0,2100.0,65.0,4200.0,41.0,4.0,630166-9,518441,0,...,1.333333,1346589.0,2.0,35.0,2,0.0,4.196267e-49,0.0,1.0,3.837098e-18
1,630384-9/797231,4800.0,1600.0,48.0,4800.0,26.0,4.0,630384-9,797231,0,...,1.418060,1395385.0,3.0,25.0,2,0.0,5.182819e-46,0.0,1.0,1.397898e-17
2,632404-65/518441,4200.0,2100.0,36.0,4200.0,12.0,1.0,632404-65,518441,0,...,1.333333,1346589.0,2.0,35.0,2,0.0,4.196267e-49,0.0,1.0,3.837098e-18
3,634941-27/826030,8700.0,2202.0,55.0,8704.0,32.0,3.0,634941-27,826030,0,...,1.850794,1491860.0,4.0,35.0,2,0.0,2.866302e-37,0.0,1.0,1.575677e-13
4,634941-45/826030,8600.0,2192.0,90.0,8704.0,69.0,5.0,634941-45,826030,0,...,1.850794,1491860.0,4.0,35.0,2,0.0,2.866302e-37,0.0,1.0,1.575677e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78679,729604-1/710671,11200.0,3767.0,58.0,11586.0,32.0,4.0,729604-1,710671,0,...,1.519685,1516068.0,3.0,25.0,2,0.0,1.738634e-51,0.0,1.0,1.172487e-20
78680,729605-1/710671,8650.0,2895.0,43.0,8709.0,22.0,5.0,729605-1,710671,0,...,1.519685,1516068.0,3.0,25.0,2,0.0,1.738634e-51,0.0,1.0,1.172487e-20
78681,729824-1/424251,4200.0,2100.0,42.0,4200.0,7.0,4.0,729824-1,424251,0,...,1.547945,1916736.0,2.0,50.0,2,0.0,5.664518e-54,0.0,1.0,2.259901e-19
78682,729882-10/702341,14800.0,4934.0,38.0,13107.0,7.0,2.0,729882-10,702341,0,...,1.519685,1414671.0,3.0,30.0,2,1.0,2.732319e-51,0.0,1.0,1.668964e-20
